In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.utils.data_utils import get_file
from keras.layers import Dropout # dropout crew 4 lyf
from IPython.display import clear_output # clear plot after each epoch
import tensorflow as tf
from tensorflow import keras as keras
import numpy as np
import random
import sys
import io
import os
import h5py
import matplotlib.pyplot as plt # fancy plots
from matplotlib import pyplot as plt
from datetime import datetime # so we know how much time we've wasted
from gtts import gTTS # tts things
import wave
from pydub import AudioSegment

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
## use nietzsche, snoop

source = 'nietzsche'
sfw = False # remove ~~bad words~~


## load a model for testing
model_path = os.path.join('trained_models',source+'.h5') # load external file
model = load_model(model_path)
print('Loaded', model_path)

## load the corpus
path = os.path.join('data',source+'.txt') # path to the corpus

with open(path, encoding='utf-8', errors='ignore') as f: # errors=ignore strips non utf-8 chars
    text = f.read().lower()

chars = sorted(list(set(text)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('corpus length :', len(text))
print('unique chars  :', len(chars))
print('total patterns:', len(sentences))
print('') # empty line

Loaded trained_models/nietzsche.h5
corpus length : 600901
unique chars  : 59
total patterns: 200287



In [3]:
## generate output
start_index = random.randint(0, len(text) - maxlen - 1)
sentence = text[start_index: start_index + maxlen]

temperature = 0.5 # 0.5-2ish, higher numbers give more unique output
maxChars = 400 # length of new output

seed = sentence.replace('\n','') # cut newlines
seed = seed.replace('\t','')

#print('*** seed: <', seed ,'>') # print the seed (text input)
#print('\n',seed, end = '')


sys.stdout = open('output.txt', 'w') # use this to output to txt instead of console

# make some words
for i in range(maxChars):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = indices_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log


In [4]:
# read generated text
f=open("output.txt", "r")
words = f.read()#.splitlines()

# clean up a few lines
words = words.replace("chorus", "")
words = words.replace("verse", "")

words = words.replace(" im ", " i'm ") # spaces give the whole word only

if(sfw):
    words = words.replace("fuck", "woof") # replace fuck with woof
    words = words.replace("nigga", "neezy")
    
    
# pick a voice
if(source == 'nietzsche'):
    language = 'en-uk'
else:
    language = 'en' # fallback
    
# make tts mp3
text = words
speech = gTTS(text = text, lang = language, slow = False)
speech.save(source+'.mp3')

# play it (doesn't do anything in jupyter, needs run through cli)
#os.system("output.mp3") # use this in some systems
#ffplay output.mp3 # use this in linux?
#afplay output.mp3 # macos